In [13]:
import tensorflow
from tensorflow import keras
from keras.layers import Embedding, Lambda, Dense

from keras.utils import np_utils, pad_sequences

from keras.preprocessing.text import Tokenizer


In [14]:
raw_data=open('corona.txt',"r")

corona_data=[text for text in raw_data if text.count(' ')>=2]

In [15]:
vectorize=Tokenizer()

In [16]:
vectorize.fit_on_texts(corona_data)

In [17]:
corona_data=vectorize.texts_to_sequences(corona_data)

In [18]:
total_vocab=sum(len(s) for s in corona_data)

In [19]:
print(total_vocab)

198


In [21]:
window_size=2

In [29]:
def cbow(data,total_vocab,window_size):
    total_len=window_size*2
    for text in data:
        text_len=len(text)
        for idx,word in enumerate(text):
            context_words=[]
            target=[]
            begin=idx-window_size
            end=idx+window_size+1
            context_words.append([text[i] for i in range(begin,end) if 0<=i<text_len and i!=idx])
            target.append(word)
            contextual=pad_sequences(context_words,maxlen=total_len)
            final_target=np_utils.to_categorical(target,total_vocab)
            
            yield(contextual,final_target)
            

In [30]:
from keras.models import Sequential

In [31]:
import keras.backend as k

In [34]:
model= Sequential(
[Embedding(total_vocab,100),Lambda(lambda x: k.mean(x,axis=1)),Dense(total_vocab,activation='softmax')])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         19800     
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 198)               19998     
                                                                 
Total params: 39,798
Trainable params: 39,798
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [39]:
for i in range(10):
    cost=0
    for x,y in cbow(corona_data,total_vocab,window_size):
        cost=cost+model.train_on_batch(x,y)
    print("EPOCH ",i,cost)

EPOCH  0 23.47691055200994
EPOCH  1 21.242812760174274
EPOCH  2 19.26554119028151
EPOCH  3 17.483465320896357
EPOCH  4 15.875002290122211
EPOCH  5 14.422214939258993
EPOCH  6 13.108846648130566
EPOCH  7 11.920415916480124
EPOCH  8 10.844091372797266
EPOCH  9 9.86852996214293
